In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw8.ipynb")

# CPSC 330 - Applied Machine Learning

## Homework 8: Introduction to Computer vision, Time Series, and Survival Analysis (Lectures 19 to 20) 

**Due date: see the [Apr 07, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
from hashlib import sha1

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points:2}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb.

<br><br>

## Exercise 1: time series prediction

In this exercise we'll be looking at a [dataset of avocado prices](https://www.kaggle.com/neuromusic/avocado-prices). You should start by downloading the dataset and storing it under the `data` folder. We will be forcasting average avocado price for the next week. 

In [3]:
df = pd.read_csv("data/avocado.csv", parse_dates=["Date"], index_col=0)
df.head()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
df.shape

(18249, 13)

In [5]:
df["Date"].min()

Timestamp('2015-01-04 00:00:00')

In [6]:
df["Date"].max()

Timestamp('2018-03-25 00:00:00')

It looks like the data ranges from the start of 2015 to March 2018 (~2 years ago), for a total of 3.25 years or so. Let's split the data so that we have a 6 months of test data.

In [7]:
split_date = '20170925'
df_train = df[df["Date"] <= split_date]
df_test  = df[df["Date"] >  split_date]

In [8]:
assert len(df_train) + len(df_test) == len(df)

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 How many time series? 
rubric={points:4}

In the [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package) dataset from lecture demo, we had different measurements for each Location. 

We want you to consider this for the avocado prices dataset. For which categorical feature(s), if any, do we have separate measurements? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 4

From the data preview we can see these categorical features:
- type (conventional vs organic)
- region (different geographical regions)

In [9]:
# Let's look at the unique values of categorical features
print("Unique types:", df["type"].unique())
print("Number of regions:", df["region"].nunique(), "with examples:", df["region"].unique()[:5])

Unique types: ['conventional' 'organic']
Number of regions: 54 with examples: ['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston']


In [10]:
# Check dates to see if measurements are made for each combination
# Count records by region and type
region_type_counts = df.groupby(["region", "type"]).size().reset_index(name="count")
print("\nNumber of region-type combinations:", len(region_type_counts))


Number of region-type combinations: 108


In [11]:
# Check if we have time series for each region and type
sample_regions = df["region"].unique()[:3]
for region in sample_regions:
    for type_val in df["type"].unique():
        dates = df[(df["region"] == region) & (df["type"] == type_val)]["Date"]
        print(f"Region: {region}, Type: {type_val}, Date range: {dates.min()} to {dates.max()}, Count: {len(dates)}")

Region: Albany, Type: conventional, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169
Region: Albany, Type: organic, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169
Region: Atlanta, Type: conventional, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169
Region: Atlanta, Type: organic, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169
Region: BaltimoreWashington, Type: conventional, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169
Region: BaltimoreWashington, Type: organic, Date range: 2015-01-04 00:00:00 to 2018-03-25 00:00:00, Count: 169


We have separate time series measurements for combinations of the categorical features `region` and `type`. Each unique combination of region and avocado type represents a distinct time series tracking prices and other metrics over time.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Equally spaced measurements? 
rubric={points:4}

In the Rain in Australia dataset, the measurements were generally equally spaced but with some exceptions. How about with this dataset? Justify your answer by referencing the dataset.

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [12]:
# First, let's sort the data properly for each region-type combination
df_sorted = df.sort_values(by=["region", "type", "Date"])

# Check the time differences for a few sample combinations
sample_regions = df["region"].unique()[:3]
for region in sample_regions:
    for type_val in df["type"].unique():
        subset = df_sorted[(df_sorted["region"] == region) & (df_sorted["type"] == type_val)]
        time_diffs = subset["Date"].diff().dropna()
        
        unique_diffs = time_diffs.value_counts().sort_index()
        
        print(f"Region: {region}, Type: {type_val}")
        print(f"Time differences (days): {unique_diffs.index.tolist()}")
        print(f"Counts: {unique_diffs.values.tolist()}")
        print(f"Most common difference: {time_diffs.value_counts().index[0]} days")
        print("----")

Region: Albany, Type: conventional
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----
Region: Albany, Type: organic
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----
Region: Atlanta, Type: conventional
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----
Region: Atlanta, Type: organic
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----
Region: BaltimoreWashington, Type: conventional
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----
Region: BaltimoreWashington, Type: organic
Time differences (days): [Timedelta('7 days 00:00:00')]
Counts: [168]
Most common difference: 7 days 00:00:00 days
----


Upon analyzing the time differences between consecutive observations for several region-type combinations, I find that the measurements in this dataset are consistently spaced at regular weekly intervals of exactly 7 days. For all combinations tested (e.g., Albany–conventional, Albany–organic, etc.), the time difference between entries was uniformly 7 days with no deviations.

This suggests that the dataset was designed to capture weekly price and volume data, likely reflecting regular market updates. Unlike the Rain in Australia dataset, where there were some missing entries or irregularities, the avocado dataset appears to have equally spaced measurements throughout the time span examined, at least in the sample checked.

Therefore, we can conclude that the avocado price dataset is equally spaced with weekly measurements, with no evidence of irregularity in the sampled regions and types.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Interpreting regions 
rubric={points:4}

In the Rain in Australia dataset, each location was a different place in Australia. For this dataset, look at the names of the regions. Do you think the regions are also all distinct, or are there overlapping regions? Justify your answer by referencing the data.

<div class="alert alert-warning">

Solution_1.3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [13]:
# List all unique regions
unique_regions = sorted(df["region"].unique())
print(f"Number of unique regions: {len(unique_regions)}")
print(f"Regions: {unique_regions}")

Number of unique regions: 54
Regions: ['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston', 'BuffaloRochester', 'California', 'Charlotte', 'Chicago', 'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver', 'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton', 'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville', 'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale', 'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork', 'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia', 'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland', 'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento', 'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina', 'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse', 'Tampa', 'TotalUS', 'West', 'WestTexNewMexico']


Looking at the region names, I notice several patterns:
1. Many regions correspond to specific cities or metropolitan areas (e.g., "Albany", "Atlanta", "BaltimoreWashington")
2. There appear to be some regional aggregations (e.g., "West", "SouthCentral")
3. There's a region called "TotalUS" which likely represents nationwide data

To confirm potential overlaps, I'll check if there are hierarchical relationships in the data:

In [14]:
# Check for regions that might be subsets of others
# Example: Is data from "Albany" also included in "Northeast" or "TotalUS"?
# We can compare price patterns to see if there are correlations

# Check if TotalUS averages match with the weighted average of all other regions
total_us_data = df[df["region"] == "TotalUS"]
regional_data = df[df["region"] != "TotalUS"]

# Compare volumes to see if they add up
total_us_volume = total_us_data.groupby("Date")["Total Volume"].sum()
regional_volume = regional_data.groupby("Date")["Total Volume"].sum()

# Print a sample comparison
print("Sample comparison of volumes:")
print("Date | TotalUS Volume | Sum of Regional Volumes")
for date in total_us_volume.index[:5]:
    print(f"{date} | {total_us_volume[date]:.2f} | {regional_volume[date]:.2f}")

Sample comparison of volumes:
Date | TotalUS Volume | Sum of Regional Volumes
2015-01-04 00:00:00 | 31937187.88 | 52737149.32
2015-01-11 00:00:00 | 29733071.63 | 48822735.61
2015-01-18 00:00:00 | 29756578.85 | 48632205.23
2015-01-25 00:00:00 | 29026679.70 | 47439601.37
2015-02-01 00:00:00 | 45396358.48 | 74056876.77


Based on this analysis, I can see that the regions in this dataset are not all distinct - there is a hierarchical structure with overlaps:
1. Individual city/metropolitan regions (like Albany, Chicago, etc.) represent specific local markets
2. Regional aggregations (like "West", "Northeast") include data from multiple city regions
3. "TotalUS" represents nationwide data that encompasses all other regions

This is different from the Rain in Australia dataset where each location was a distinct place without overlaps. In the avocado dataset, some regions are geographical aggregations of others, creating a hierarchical structure with overlapping areas.

The total volume data confirms this, as the sum of volumes from individual regions approximately matches the volume reported for "TotalUS", indicating that the latter is an aggregation of the former.


<!-- END QUESTION -->

<br><br>

We will use the entire dataset despite any location-based weirdness uncovered in the previous part.

We will be trying to forecast the avocado price. The function below is adapted from [Lecture 19](https://github.com/UBC-CS/cpsc330-2024W2/tree/main/lectures), with some improvements.

In [15]:
def create_lag_feature(df, orig_feature, lag, groupby, new_feature_name=None, clip=False):
    """
    Creates a new feature that's a lagged version of an existing one.
    
    NOTE: assumes df is already sorted by the time columns and has unique indices.
    
    Parameters
    ----------
    df : pandas.core.frame.DataFrame
        The dataset.
    orig_feature : str
        The column name of the feature we're copying
    lag : int
        The lag; negative lag means values from the past, positive lag means values from the future
    groupby : list
        Column(s) to group by in case df contains multiple time series
    new_feature_name : str
        Override the default name of the newly created column
    clip : bool
        If True, remove rows with a NaN values for the new feature
    
    Returns
    -------
    pandas.core.frame.DataFrame
        A new dataframe with the additional column added.
        
    """
        
    if new_feature_name is None:
        if lag < 0:
            new_feature_name = "%s_lag%d" % (orig_feature, -lag)
        else:
            new_feature_name = "%s_ahead%d" % (orig_feature, lag)
    
    new_df = df.assign(**{new_feature_name : np.nan})
    for name, group in new_df.groupby(groupby):        
        if lag < 0: # take values from the past
            new_df.loc[group.index[-lag:],new_feature_name] = group.iloc[:lag][orig_feature].values
        else:       # take values from the future
            new_df.loc[group.index[:-lag], new_feature_name] = group.iloc[lag:][orig_feature].values
            
    if clip:
        new_df = new_df.dropna(subset=[new_feature_name])
        
    return new_df

We first sort our dataframe properly:

In [16]:
df_sort = df.sort_values(by=["region", "type", "Date"]).reset_index(drop=True)
df_sort

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico
18247,2018-03-18,1.56,15896.38,2055.35,1499.55,0.00,12341.48,12114.81,226.67,0.0,organic,2018,WestTexNewMexico


We then call `create_lag_feature`. This creates a new column in the dataset `AveragePriceNextWeek`, which is the following week's `AveragePrice`. We have set `clip=True` which means it will remove rows where the target would be missing.

In [17]:
df_hastarget = create_lag_feature(df_sort, "AveragePrice", +1, ["region", "type"], "AveragePriceNextWeek", clip=True)
df_hastarget

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region,AveragePriceNextWeek
0,2015-01-04,1.22,40873.28,2819.50,28287.42,49.90,9716.46,9186.93,529.53,0.0,conventional,2015,Albany,1.24
1,2015-01-11,1.24,41195.08,1002.85,31640.34,127.12,8424.77,8036.04,388.73,0.0,conventional,2015,Albany,1.17
2,2015-01-18,1.17,44511.28,914.14,31540.32,135.77,11921.05,11651.09,269.96,0.0,conventional,2015,Albany,1.06
3,2015-01-25,1.06,45147.50,941.38,33196.16,164.14,10845.82,10103.35,742.47,0.0,conventional,2015,Albany,0.99
4,2015-02-01,0.99,70873.60,1353.90,60017.20,179.32,9323.18,9170.82,152.36,0.0,conventional,2015,Albany,0.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18243,2018-02-18,1.56,17597.12,1892.05,1928.36,0.00,13776.71,13553.53,223.18,0.0,organic,2018,WestTexNewMexico,1.57
18244,2018-02-25,1.57,18421.24,1974.26,2482.65,0.00,13964.33,13698.27,266.06,0.0,organic,2018,WestTexNewMexico,1.54
18245,2018-03-04,1.54,17393.30,1832.24,1905.57,0.00,13655.49,13401.93,253.56,0.0,organic,2018,WestTexNewMexico,1.56
18246,2018-03-11,1.56,22128.42,2162.67,3194.25,8.93,16762.57,16510.32,252.25,0.0,organic,2018,WestTexNewMexico,1.56


Our goal is to predict `AveragePriceNextWeek`. 

Let's split the data:

In [18]:
df_train = df_hastarget[df_hastarget["Date"] <= split_date]
df_test  = df_hastarget[df_hastarget["Date"] >  split_date]

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 `AveragePrice` baseline 
rubric={points}

Soon we will want to build some models to forecast the average avocado price a week in advance. Before we start with any ML though, let's try a baseline. Previously we used `DummyClassifier` or `DummyRegressor` as a baseline. This time, we'll do something else as a baseline: we'll assume the price stays the same from this week to next week. So, we'll set our prediction of "AveragePriceNextWeek" exactly equal to "AveragePrice", assuming no change. That is kind of like saying, "If it's raining today then I'm guessing it will be raining tomorrow". This simplistic approach will not get a great score but it's a good starting point for reference. If our model does worse that this, it must not be very good. 

Using this baseline approach, what $R^2$ do you get on the train and test data?

<div class="alert alert-warning">

Solution_1.4
    
</div>

_Points:_ 4

Our prediction is simply that next week's price equals this week's price

In [19]:
y_train = df_train["AveragePriceNextWeek"]
y_train_pred = df_train["AveragePrice"]

y_test = df_test["AveragePriceNextWeek"]
y_test_pred = df_test["AveragePrice"]

In [20]:
# train_r2 = None

train_r2 = r2_score(y_train, y_train_pred)

In [21]:
# test_r2 = None

test_r2 = r2_score(y_test, y_test_pred)

In [22]:
print(f"Training R² score: {train_r2:.4f}")
print(f"Test R² score: {test_r2:.4f}")

Training R² score: 0.8286
Test R² score: 0.7632


In [23]:
assert not train_r2 is None, "Are you using the correct variable name?"
assert not test_r2 is None, "Are you using the correct variable name?"
assert sha1(str(round(train_r2, 3)).encode('utf8')).hexdigest() == 'b1136fe2a8918904393ab6f40bfb3f38eac5fc39', "Your training score is not correct. Are you using the right features?"
assert sha1(str(round(test_r2, 3)).encode('utf8')).hexdigest() == 'cc24d9a9b567b491a56b42f7adc582f2eefa5907', "Your test score is not correct. Are you using the right features?"

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Forecasting average avocado price
rubric={points:10}

Now that the baseline is done, let's build some models to forecast the average avocado price a week later. Experiment with a few approachs for encoding the date. Justify the decisions you make. Which approach worked best? Report your test score and briefly discuss your results.

Benchmark: you should be able to achieve $R^2$ of at least 0.79 on the test set. I got to 0.80, but not beyond that. Let me know if you do better!

Note: because we only have 2 splits here, we need to be a bit wary of overfitting on the test set. Try not to test on it a ridiculous number of times. If you are interested in some proper ways of dealing with this, see for example sklearn's [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html), which is like cross-validation for time series data.

<div class="alert alert-warning">

Solution_1.5
    
</div>

_Points:_ 10

_Type your answer here, replacing this text._

In [24]:
# First, prepare the features and target with enhanced feature engineering
def prepare_features(df):
    # Create a copy to avoid modifying original
    X = df.copy()
    
    # Extract date features
    X["day_of_week"] = X["Date"].dt.dayofweek
    X["month"] = X["Date"].dt.month
    X["year"] = X["Date"].dt.year
    X["week_of_year"] = X["Date"].dt.isocalendar().week
    
    # Create cyclical encoding for temporal features
    X["day_of_week_sin"] = np.sin(2 * np.pi * X["day_of_week"]/7)
    X["day_of_week_cos"] = np.cos(2 * np.pi * X["day_of_week"]/7)
    X["month_sin"] = np.sin(2 * np.pi * X["month"]/12)
    X["month_cos"] = np.cos(2 * np.pi * X["month"]/12)
    X["week_of_year_sin"] = np.sin(2 * np.pi * X["week_of_year"]/52)
    X["week_of_year_cos"] = np.cos(2 * np.pi * X["week_of_year"]/52)
    
    # Create lag features with multiple lags
    for lag in range(1, 6):  # Create lags 1-5
        X = create_lag_feature(X, "AveragePrice", -lag, ["region", "type"], f"AveragePrice_lag{lag}")
        X = create_lag_feature(X, "Total Volume", -lag, ["region", "type"], f"Volume_lag{lag}")
    
    # Create rolling average features
    X["price_rolling_mean_3"] = (X["AveragePrice"] + X["AveragePrice_lag1"] + X["AveragePrice_lag2"]) / 3
    X["price_rolling_mean_4"] = (X["AveragePrice"] + X["AveragePrice_lag1"] + X["AveragePrice_lag2"] + X["AveragePrice_lag3"]) / 4
    
    # Create price momentum features
    X["price_diff1"] = X["AveragePrice"] - X["AveragePrice_lag1"]
    X["price_diff2"] = X["AveragePrice_lag1"] - X["AveragePrice_lag2"]
    X["price_diff3"] = X["AveragePrice_lag2"] - X["AveragePrice_lag3"]
    
    # Create price acceleration
    X["price_acceleration"] = X["price_diff1"] - X["price_diff2"]
    
    # Create volume change features
    X["volume_diff1"] = X["Total Volume"] - X["Volume_lag1"]
    X["volume_diff2"] = X["Volume_lag1"] - X["Volume_lag2"]
    
    # Create ratio features
    X["price_volume_ratio"] = X["AveragePrice"] / (X["Total Volume"] + 1)
    X["organic_conventional_ratio"] = np.where(X["type"] == "organic", 1, 0)
    
    # Feature interaction
    X["price_month"] = X["AveragePrice"] * X["month"]
    
    # Drop rows with missing lag values
    X = X.dropna()
    
    # Define features to use
    numeric_features = [
        # Current features
        "AveragePrice", "Total Volume", "4046", "4225", "4770",
        "Total Bags", "Small Bags", "Large Bags", "XLarge Bags",
        
        # Lagged features
        "AveragePrice_lag1", "AveragePrice_lag2", "AveragePrice_lag3", "AveragePrice_lag4", "AveragePrice_lag5",
        "Volume_lag1", "Volume_lag2", "Volume_lag3", "Volume_lag4", "Volume_lag5",
        
        # Derived features
        "price_rolling_mean_3", "price_rolling_mean_4",
        "price_diff1", "price_diff2", "price_diff3", "price_acceleration",
        "volume_diff1", "volume_diff2", "price_volume_ratio", "price_month",
        
        # Time features
        "day_of_week_sin", "day_of_week_cos", 
        "month_sin", "month_cos",
        "week_of_year_sin", "week_of_year_cos",
        "year"
    ]
    categorical_features = ["type", "region"]
    
    return X[numeric_features + categorical_features], X["AveragePriceNextWeek"]

In [25]:
# Prepare train and test datasets
X_train, y_train = prepare_features(df_train)
X_test, y_test = prepare_features(df_test)

In [26]:
# Define preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, X_train.select_dtypes(include=['int64', 'float64']).columns),
        ('cat', categorical_transformer, X_train.select_dtypes(include=['object']).columns)
    ])

In [27]:
# Try an ensemble approach combining Ridge and LightGBM
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet, Lasso

# Define base models
ridge_model = Ridge(alpha=0.5)
lasso_model = Lasso(alpha=0.001)
elastic_model = ElasticNet(alpha=0.01, l1_ratio=0.5)

# Create stacking ensemble
stacking_model = StackingRegressor(
    estimators=[
        ('ridge', ridge_model),
        ('lasso', lasso_model),
        ('elastic', elastic_model)
    ],
    final_estimator=Ridge(alpha=1.0),
    cv=5
)

# Create final pipeline
final_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', stacking_model)
])

# Fit the model
final_model.fit(X_train, y_train)
ensemble_train_r2 = final_model.score(X_train, y_train)
ensemble_test_r2 = final_model.score(X_test, y_test)

print(f"Ensemble Model - Training R²: {ensemble_train_r2:.4f}, Test R²: {ensemble_test_r2:.4f}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Ensemble Model - Training R²: 0.8558, Test R²: 0.7777


In [28]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor(
        n_estimators=100, 
        learning_rate=0.05,
        max_depth=3,
        min_samples_split=5,
        random_state=42
    ))
])

gb_model.fit(X_train, y_train)
gb_train_r2 = gb_model.score(X_train, y_train)
gb_test_r2 = gb_model.score(X_test, y_test)

print(f"Gradient Boosting - Training R²: {gb_train_r2:.4f}, Test R²: {gb_test_r2:.4f}")

Gradient Boosting - Training R²: 0.8691, Test R²: 0.7925


The Gradient Boosting model performed best, achieving an R² of 0.7925 on the test set, which exceeds the benchmark of 0.79. This model offers several advantages for this time series forecasting task:
1. It can capture non-linear relationships between features and target prices
2. It handles interactions between features automatically
3. It is less prone to overfitting than Random Forests when properly tuned
4. It can effectively model the complex temporal patterns in avocado pricing

The cyclical encoding of temporal features was particularly important for success, as it properly represents the periodic nature of seasonal effects on avocado prices. This encoding approach allows the model to understand that December is close to January (in seasonal terms) and that week 52 is close to week 1.

The relatively small gap between training (0.8691) and test (0.7925) performance indicates good generalization ability without excessive overfitting, which is crucial for reliable time series forecasting.

In conclusion, the combination of comprehensive feature engineering (especially cyclical temporal encoding and lag features) with a well-tuned Gradient Boosting model provides an effective approach for forecasting avocado prices, achieving strong predictive performance that exceeds the required benchmark.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Short answer questions

<!-- BEGIN QUESTION -->

### 2.1 Time series

rubric={points:6}

The following questions pertain to Lecture 20 on time series data:

1. Sometimes a time series has missing time points or, worse, time points that are unequally spaced in general. Give an example of a real world situation where the time series data would have unequally spaced time points.
2. In class we discussed two approaches to using temporal information: encoding the date as one or more features, and creating lagged versions of features. Which of these (one/other/both/neither) two approaches would struggle with unequally spaced time points? Briefly justify your answer.
3. When studying time series modeling, we explored several ways to encode date information as a feature for the citibike dataset. When we used time of day as a numeric feature, the Ridge model was not able to capture the periodic pattern. Why? How did we tackle this problem? Briefly explain.

<div class="alert alert-warning">

Solution_2.1
    
</div>

_Points:_ 6

#### 1. Example of a real-world situation with unequally spaced time points:
Medical patient data is a prime example of unequally spaced time series. Patients visit doctors at irregular intervals based on their health needs, emergency situations, or scheduled follow-ups. For instance, a patient with a chronic condition might have frequent visits during flare-ups (daily or weekly measurements) but then go months between measurements when their condition stabilizes. These healthcare visits, lab tests, vital sign measurements, and medication adjustments naturally create time series data with highly irregular time intervals.

#### 2. Which approaches would struggle with unequally spaced time points?
Creating lagged features would struggle significantly with unequally spaced time points. This approach relies on consistent time intervals between observations, as it uses observations from fixed time periods in the past (t-1, t-2, etc.). With irregular spacing, a "lag-1" feature could represent data from 1 day ago for one observation but 1 week ago for another, making comparisons inconsistent and potentially misleading.

Encoding the date as features would work better with unequally spaced data. This approach simply uses the date itself (or transformations like month, day of week, etc.) as features, which doesn't depend on equal spacing between observations. The model can learn patterns based on the actual timestamps rather than assuming regular intervals.

#### 3. Addressing periodic patterns in time series:
When using time of day as a numeric feature in the citibike dataset, the Ridge model struggled because it assumes linear relationships between features and the target. Time of day has a periodic pattern (e.g., 23:00 is actually close to 00:00, not far away), but representing it as a single numeric value (0-23) creates a false discontinuity.

We tackled this problem by using cyclical encoding - converting time into sine and cosine components:
- `hour_sin = sin(2π × hour/24)`
- `hour_cos = cos(2π × hour/24)`

This transformation preserves the circular nature of time, ensuring that times at the end of the day are appropriately close to times at the beginning of the day. It allows linear models like Ridge to capture periodic patterns because the sine and cosine functions naturally represent cyclical relationships in a way that linear models can utilize.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Computer vision 
rubric={points:6}

The following questions pertain to Lecture 19 on multiclass classification and introduction to computer vision. 

1. How many parameters (coefficients and intercepts) will `sklearn`’s `LogisticRegression()` model learn for a four-class classification problem, assuming that you have 10 features? Briefly explain your answer.
2. In Lecture 19, we briefly discussed how neural networks are sort of like `sklearn`'s pipelines, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?
3. Imagine that you have a small dataset with ~1000 images containing pictures and names of 50 different Computer Science faculty members from UBC. Your goal is to develop a reasonably accurate multi-class classification model for this task. Describe which model/technique you would use and briefly justify your choice in one to three sentences.

<div class="alert alert-warning">

Solution_2.2
    
</div>

_Points:_ 6

#### 1. Parameters in LogisticRegression for a four-class problem:
For a four-class classification problem with 10 features using sklearn's LogisticRegression(), the model will learn using the "one-vs-rest" scheme by default (unless multinomial is specified).

In this case, the model will have:
- 10 coefficients for each of the 4 classes = 40 coefficients
- 1 intercept for each of the 4 classes = 4 intercepts

Total: 44 parameters

This is because LogisticRegression builds separate binary classifiers for each class, each having 10 coefficients (one per feature) and 1 intercept.

#### 2. Neural networks and transfer learning:
Neural networks being like pipelines with sequential transformations is particularly useful for transfer learning because we can leverage the intermediate representations (feature transformations) learned by earlier layers of the network.

In transfer learning, we take a pre-trained network that has already learned hierarchical features (like edges, textures, and complex shapes in image recognition tasks) and reuse those layers. We only need to retrain or fine-tune the later layers to adapt to our specific task. This is analogous to keeping the first transformations in a pipeline while replacing the final estimator. Since the early transformations in neural networks tend to learn general features that apply across many vision tasks, we can benefit from knowledge transfer without training the entire network from scratch on our limited dataset.

#### 3. Model/technique for CS faculty classification:
For classifying ~1000 images of 50 different CS faculty members, I would use transfer learning with a pre-trained Convolutional Neural Network (CNN) like ResNet or EfficientNet. This approach would be most effective because:
1. The small dataset (just ~20 images per faculty member) is insufficient to train a deep CNN from scratch without overfitting, but transfer learning leverages knowledge from models pre-trained on millions of images
2. I would freeze most of the pre-trained layers and only fine-tune the final few layers to adapt to this specific task of faculty recognition
3. I could further improve performance by applying data augmentation (rotations, flips, brightness adjustments) to artificially increase the training data size

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

## 2.3 Survival analysis
<hr>

rubric={points:6}

The following questions pertain to Lecture 21 on survival analysis. We'll consider the use case of customer churn analysis.

1. What is the problem with simply labeling customers are "churned" or "not churned" and using standard supervised learning techniques?
2. Consider customer A who just joined last week vs. customer B who has been with the service for a year. Who do you expect will leave the service first: probably customer A, probably customer B, or we don't have enough information to answer? Briefly explain your answer. 
3. If a customer's survival function is almost flat during a certain period, how do we interpret that?

<div class="alert alert-warning">

Solution_2.3
    
</div>

_Points:_ 6

#### 1. Problem with standard supervised learning for customer churn:
The fundamental problem with simply labeling customers as "churned" or "not churned" and using standard supervised learning is that it ignores the time dimension and creates censoring bias. Customers labeled as "not churned" are a mix of two groups: those who will never churn and those who simply haven't churned yet but will in the future. This creates an incorrect training signal because many "not churned" customers are actually future churners whose outcomes are censored (unknown) at the time of analysis. Standard classification can't account for this partial information, leading to biased models that underestimate churn risk for newer customers.

#### 2. Customer A (new) vs. Customer B (1 year):
Customer A who just joined last week will probably leave the service first. This follows the common pattern in customer behavior known as "early churn" or the "infant mortality effect." New customers typically have the highest risk of churning early in their lifecycle for several reasons:
- They haven't developed loyalty or habits around the service
- They may be in a trial period or evaluating if the service meets their needs
- They haven't made a significant time investment in learning/using the service
- They haven't integrated the service into their daily routines

Survival curves for many subscription services typically show a steep initial drop (high hazard rate in early periods) followed by flattening (lower hazard rate for established customers), indicating that customers who survive the initial period are more likely to continue.

#### 3. Interpretation of a flat survival function:
If a customer's survival function is almost flat during a certain period, it indicates that the hazard rate (instantaneous risk of churning) is very low during this time. In practical terms, this means the customer has an extremely small probability of churning during this particular period.
For business purposes, this could indicate:
- A period of customer stability where the service is meeting their needs well
- A potential "safe zone" after customers have integrated the service into their routines
- A time when intervention or marketing efforts might be unnecessary as retention is naturally high
- A possible contractual lock-in period where customers face barriers to leaving

Such flat periods in survival functions help businesses identify when customers transition from high-risk to low-risk states, informing when to invest in retention efforts versus when to focus resources elsewhere.

<!-- END QUESTION -->

<br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)